In [3]:
# Import packages and load the data
import numpy as np
import numpy.linalg as lin
import pandas as pd
import scipy.stats as sts
import scipy.integrate as intgr
import scipy.optimize as opt
import matplotlib
import matplotlib.pyplot as plt

# This next command is specifically for Jupyter Notebook
%matplotlib notebook

df = pd.read_csv('sick.txt', skiprows = [0], names = ['sick', 'age', 'children', 'avgtemp_winter'])

In [33]:
data_array = df.values
print(data_array[0][1])
print(data_array[0])

data_array[:,1]

57.47
[  1.67  57.47   3.04  54.1 ]


array([ 57.47,  26.77,  41.85,  51.27,  44.22,  44.67,  27.92,  44.87,
        53.68,  39.63,  44.73,  41.05,  30.44,  37.77,  30.46,  46.54,
        51.92,  15.46,  59.02,  45.63,  42.65,  28.09,  35.63,  54.79,
        42.76,  38.5 ,  36.14,  20.16,  40.3 ,  55.48,  46.37,  52.69,
        55.31,  52.21,  38.91,  26.21,  42.66,  37.89,  46.45,  38.49,
        41.43,  32.41,  48.77,  33.9 ,  48.81,  74.89,  36.7 ,  44.9 ,
        54.07,  44.15,  47.06,  41.47,  63.36,  61.14,  50.63,  43.83,
        35.15,  48.5 ,  45.68,  17.01,  56.44,  51.19,  43.76,  34.55,
        28.41,  21.15,  51.08,  54.8 ,  24.01,  43.18,  46.43,  37.01,
        37.91,  37.58,  28.65,  38.59,  34.36,  24.51,  65.  ,  49.24,
        29.84,  24.98,  38.33,  30.11,  44.88,  52.1 ,  17.91,  42.38,
        28.25,  31.12,  49.13,  37.03,  30.93,  31.4 ,  46.23,  40.97,
        34.33,  40.98,  30.61,  22.64,  33.03,  44.19,  12.99,  20.63,
        32.11,  25.34,  48.41,  48.2 ,  47.22,  34.1 ,  37.08,  49.81,
      

In [38]:
sick_df = df['sick']
sick_array = sick_df.values

age_df = df['age']
age_array = age_df.values

children_df = df['children']
children_array = children_df.values

avgtemp_winter_df = df['avgtemp_winter']
winterT_array = avgtemp_winter_df.values

data_array.shape
val = 2 * data_array[1,1]
val

53.539999999999999

In [88]:
def model_moments(data, beta0, beta1, beta2, beta3):
    mod_mom_array = np.ones(200,)
    for x in range(200):
        mod_mom_array[x,] = beta0 + beta1 * data[x,1] + beta2 * data[x,2] + beta3 * data[x,3] 
                
    return mod_mom_array

#mod_mom_array = model_moments(data_array, beta0, beta1, beta2, beta3)
#dat_mom_array = data_array[:,0]

#df['err'] = beta0 + beta1 * data_array[:,1] + beta2 * data_array[:,2] + beta3 * data_array[:,3] 
#print(df['err'])
#print(mod_mom_array)

In [106]:
def error_vec(data, beta0, beta1, beta2, beta3):
    
    dat_mom_array = data[:,0]
    mod_mom_array = model_moments(data, beta0, beta1, beta2, beta3)
    err_vec = mod_mom_array - dat_mom_array
    
    return err_vec

err_vec_array = error_vec(data_array, -1, -1, 2, 1)
err_vec_array

array([  0.04,  10.46,  -7.24,   4.22,   4.67,  24.71,   3.12,   8.13,
         2.69,  -0.59,  -1.77,  16.13,  11.43,  10.44,   4.79,  -0.33,
        14.83,  16.88,  -0.54,  -1.27,   6.57,  17.04,  11.84,   3.92,
         0.7 ,  14.44,  -3.  ,  10.49,  14.29,   0.91,   7.16,  -4.36,
         5.22, -17.88,   3.19,   1.1 ,  -6.3 ,   6.12,  17.23,   4.31,
         2.29, -10.58,   0.04,  16.98,   8.13, -28.33,  -6.91,  -1.04,
        -8.  ,   0.06,   5.34,   9.06,  -7.  ,   4.56,   2.74,  -5.54,
         9.56,  -5.76,   3.7 ,   8.63,  -3.43,   5.97,  10.89,  -4.08,
        18.04,   4.41,  10.25,  -8.09,   0.92,  14.41,   4.36,   7.16,
        -8.47,   9.73,   3.66,  -0.7 ,   7.17,  14.3 ,  -0.58,  -1.35,
         6.4 ,   4.43,  14.27,  13.45,   5.23, -16.08,  21.89,  12.96,
         3.41,  21.85,   1.93,  -6.23,  22.85,   1.28,  -6.68,  -7.7 ,
         9.49,  -3.61,  -1.86,  17.11,   2.82,   9.55,  16.2 ,  18.32,
         3.19,  23.94, -10.28,  -2.84,   1.58,  -4.89,  24.54,   0.7 ,
      

In [94]:
def criterion(params, *args):
    beta0, beta1, beta2, beta3 = params
    data, W = args
    err_vec_array = error_vec(data_array, beta0, beta1, beta2, beta3)
    crit_val = np.dot(np.dot(err_vec_array.T, W), err_vec_array) 
    
    return crit_val

In [102]:
beta0_init, beta1_init, beta2_init, beta3_init = (-1,-1, 2, 1)
params_init = np.array([beta0_init, beta1_init, beta2_init, beta3_init])
W_hat = np.eye(200)
gmm_args = (data_array, W_hat)
results = opt.minimize(criterion, params_init, args=(gmm_args),
                       method='SLSQP')
beta0_GMM, beta1_GMM, beta2_GMM, beta3_GMM = results.x
print('beta0_GMM=', beta0_GMM, 'beta1_GMM=', beta1_GMM, 'beta2_GMM=', beta2_GMM, 'beta3_GMM=', beta3_GMM)
print('beta0_init=', beta0_init, 'beta1_init=', beta1_init, 'beta2_init=', beta2_init, 'beta3_init=', beta3_init)

beta0_GMM= 0.25164447235 beta1_GMM= 0.0129333849031 beta2_GMM= 0.400501883538 beta3_GMM= -0.0099916546916
beta0_init= -1 beta1_init= -1 beta2_init= 2 beta3_init= 1


In [103]:
results

     fun: 0.0018212898913348983
     jac: array([  1.55335438e-05,   2.38940325e-03,   6.99157099e-05,
         2.61543067e-03,   0.00000000e+00])
 message: 'Optimization terminated successfully.'
    nfev: 51
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([ 0.25164447,  0.01293338,  0.40050188, -0.00999165])

In [107]:
err = error_vec(data_array, beta0_GMM, beta1_GMM, beta2_GMM, beta3_GMM).reshape(200,)
print('Error vector=', err)

Error vector= [  1.90330987e-03   3.37838402e-03   4.53620260e-03   1.01555046e-03
   4.30692564e-03   4.55352437e-03  -4.28643647e-03  -1.30867194e-03
   5.99667518e-03  -3.51051220e-03   1.39241863e-03   2.25531085e-03
  -5.57592812e-03  -2.68948489e-03  -7.20510948e-04   2.38752452e-03
   7.60900201e-04  -2.72616303e-03  -1.92253831e-03   3.12492165e-03
  -2.72171936e-03   2.52947515e-03  -3.59769757e-03  -4.05896403e-03
   3.98036941e-03  -1.67510835e-03  -2.18773987e-03   3.21600214e-03
   2.48003624e-03   1.49585731e-03  -1.34350171e-03  -2.40996438e-03
   6.00644502e-04   2.02594525e-03   1.41430335e-03   6.12658595e-03
  -5.15322891e-03   3.64279185e-03   1.95511701e-03  -1.69502702e-03
   5.08131816e-03  -1.03410453e-03  -7.27113133e-04   4.06000746e-03
  -3.81622546e-03   2.65249436e-03   6.83331528e-04   1.31809601e-03
   3.14240856e-03   4.07459977e-03   2.95884168e-03  -3.45250913e-03
  -4.87376279e-03  -1.21456452e-03   2.33716075e-03   3.64037033e-03
  -5.93309260e-03   